# Branch and bound

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

import numpy as np
from IPython.core.display_functions import display
from teaching_optimization.branch_and_bound.branch_and_bound_milp import LinearProblem
from teaching_optimization.branch_and_bound.solution import Solution
from teaching_optimization.linear_constraints import (
    SignVariable,
    Variable,
    Term,
    SignConstraint,
    Constraint,
    AllConstraints,
)


The objective of this exercise is to implement the branch and bound algorithm and solve mixed integer linear
optimization problems.

Consider the optimization problem :
$$
\min x_1 - 2 x_2
$$
subject to
$$
\begin{array}{rcl}
-4 x_1 + 6 x_2 & \leq & 9 \\
x_1 + x_2 &\leq & 4 \\
x_1, x_2 &\geq& 0 \\
x_1, x_2 & \in & \mathbb{N}.
\end{array}
$$

We first encode the relaxation as linear optimization problem.

Coefficients of the objective function

In [ ]:
coefficients = np.array([1, -2])


Non negative variables

In [ ]:
x1 = Variable('x1', sign=SignVariable.NON_NEGATIVE)
x2 = Variable('x2', sign=SignVariable.NON_NEGATIVE)


First constraint.

In [ ]:
term_1_1 = Term(
    variable=x1,
    coefficient=-4,
)
term_1_2 = Term(variable=x2, coefficient=6)
constraint_1 = Constraint(
    name='first constraint',
    left_hand_side=[term_1_1, term_1_2],
    sign=SignConstraint.LESSER_OR_EQUAL,
    right_hand_side=9,
)


Second constraint

In [ ]:
term_2_1 = Term(
    variable=x1,
    coefficient=1,
)
term_2_2 = Term(variable=x2, coefficient=1)
constraint_2 = Constraint(
    name='second constraint',
    left_hand_side=[term_2_1, term_2_2],
    sign=SignConstraint.LESSER_OR_EQUAL,
    right_hand_side=4,
)


All constraints.

In [ ]:
all_constraints = AllConstraints(constraints=[constraint_1, constraint_2])
print(all_constraints)


The definition of the problem gathers all the data defined above, and provides a name.

In [ ]:
the_problem: LinearProblem = LinearProblem(
    objective_coefficients=coefficients, constraints=all_constraints, the_name='P'
)


If you print the problem, you obtain its name:

In [ ]:
print(the_problem)


You can also print the constraints.

In [ ]:
print(the_problem.constraints)


Using this object, you can calculate a bound and obtain a tuple with

- an object of type ``Solution``,
- a boolean.

The object of type ``Solution`` provides both the solution and the value of the objective function.
If it is ``None``, it means that the problem is infeasible.
The boolean is True if an integer solution has been found, and False otherwise.

In [ ]:
calculated_bound = the_problem.bound()
if calculated_bound is not None:
    the_solution, is_integer = calculated_bound


The solution

In [ ]:
display(the_solution)


Is integer?

In [ ]:
display(is_integer)


It also provides the "branching", that is, a list of subproblems of the same type.
It means that each of the subproblems have a function ``branch`` and a function ``bound``.

In [ ]:
the_subproblems: list[LinearProblem] = the_problem.branch()

for subproblem in the_subproblems:
    print(f'Subproblem {subproblem}')
    print(subproblem.constraints)


Now, we ask you to write a recursive function that takes a subproblem as input and solves it
using the branch and bound method.

In [ ]:


def solve(a_subproblem: LinearProblem, upper_bound=np.inf, level=0) -> Solution | None:
    """

    :param a_subproblem: the subproblem to solve
    :param upper_bound: an upper bound on the optimal solution.
    :param level: level of recursion. Used only for printing the output.
    :return: the optimal solution if the problem has been solved. None if it is infeasible, or if there is no point
        solving it as it will not provide the optimal solution of the original problem.
    """
    # Use the same trick as below for your own printing statements
    # so that the recursive calls are properly indented.
    indentation = ' ' * level * 4
    print(f'{indentation}**** Solving problem {a_subproblem.name} ****')
    constraints = a_subproblem.constraints.report()
    for constraint in constraints:
        print(f'{indentation}{constraint}')
    calculated_bound = a_subproblem.bound()
    if calculated_bound is not None:
        solution, is_integer = calculated_bound

    # At this point, we have solved the relaxation of the subproblem. Continue the implementation
    # of the algorithm.

    # If the problem is infeasible, return None
    if calculated_bound is None:
        # The subproblem is infeasible
        print(f'{indentation}{a_subproblem.name}: infeasible')
        return None

    # If the solution of the relaxation is also the solution of the integer problem,
    # we can return the solution.
    if is_integer:
        print(f'{indentation}{a_subproblem.name}: optimal solution {solution}')
        return solution

    # Otherwise, we have the solution of the relaxation.
    print(f'{indentation}{a_subproblem.name}: solution of relaxation {solution}')

    # We first need to check if the problem is worth solving or not.
    if solution.value >= upper_bound:
        """This subproblem is suboptimal. No need to solve it."""
        print(
            f'{indentation}{a_subproblem.name}: lower bound {solution.value} '
            f'larger than upper bound {upper_bound}'
        )
        return None

    # At this point, we need to solve the problem. We decompose it into subproblems.
    subproblems = a_subproblem.branch()

    best_solution = None

    # We recursively call the function for each subproblem.
    for subproblem in subproblems:
        solution: Solution | None = solve(
            a_subproblem=subproblem, upper_bound=upper_bound, level=level + 1
        )
        # If ``None``is returned, we ignore this subproblem.
        if solution is None:
            continue
        # If it is the first subproblem solved, it is also the best solution so far.
        if best_solution is None:
            best_solution = solution
        # Otherwise, update the best solution and the upper bound.
        elif solution.value < best_solution.value:
            best_solution = solution
        if best_solution.value < upper_bound:
            upper_bound = best_solution.value

    print(f'{indentation}{a_subproblem.name}: optimal solution {best_solution}')
    return best_solution



We apply this function on the original problem

In [ ]:
optimal_solution = solve(a_subproblem=the_problem)
print(optimal_solution)


## Second example.
Apply the algorithm to solve the following problem.

$$
\min  -13/4x_1-  8x_2
$$
subject to
\begin{align*}
5x_1+ 6x_2 &\leq 30,\\
-x_1+ 2x_2 &\leq 6,\\
x_1,x_2 &\in \mathbb{N}.
\end{align*}

Coefficients of the objective function

In [ ]:
coefficients = np.array([-13 / 4, -8])


Non negative variables

In [ ]:
x1 = Variable('x1', sign=SignVariable.NON_NEGATIVE)
x2 = Variable('x2', sign=SignVariable.NON_NEGATIVE)


First constraint.

In [ ]:
term_1_1 = Term(
    variable=x1,
    coefficient=5,
)
term_1_2 = Term(variable=x2, coefficient=6)
constraint_1 = Constraint(
    name='first constraint',
    left_hand_side=[term_1_1, term_1_2],
    sign=SignConstraint.LESSER_OR_EQUAL,
    right_hand_side=30,
)


Second constraint

In [ ]:
term_2_1 = Term(
    variable=x1,
    coefficient=-1,
)
term_2_2 = Term(variable=x2, coefficient=2)
constraint_2 = Constraint(
    name='second constraint',
    left_hand_side=[term_2_1, term_2_2],
    sign=SignConstraint.LESSER_OR_EQUAL,
    right_hand_side=6,
)


All constraints.

In [ ]:
all_constraints = AllConstraints(
    constraints=[constraint_1, constraint_2]
)
print(all_constraints)


The definition of the problem gathers all the data defined above, and provides a name.

In [ ]:
the_problem: LinearProblem = LinearProblem(
    objective_coefficients=coefficients, constraints=all_constraints, the_name='P'
)


We apply the algorithm.

In [ ]:
optimal_solution = solve(a_subproblem=the_problem)
print(optimal_solution)


## Third example.
Apply the algorithm to solve the following problem.

$$
\min_{x\in \mathbb{N}^2} -13 x_1 -8 x_2
$$
subject to
\begin{align*}
x_1 + 2 x_2 & \leq 10, \\
5 x_1 + 2 x_2 & \leq 20, \\
x_1, x_2 & \in \mathbb{N}.
\end{align*}

Coefficients of the objective function

In [ ]:
coefficients = np.array([-13, -8])


Non negative variables

In [ ]:
x1 = Variable('x1', sign=SignVariable.NON_NEGATIVE)
x2 = Variable('x2', sign=SignVariable.NON_NEGATIVE)


First constraint.

In [ ]:
term_1_1 = Term(
    variable=x1,
    coefficient=1,
)
term_1_2 = Term(variable=x2, coefficient=2)
constraint_1 = Constraint(
    name='first constraint',
    left_hand_side=[term_1_1, term_1_2],
    sign=SignConstraint.LESSER_OR_EQUAL,
    right_hand_side=10,
)


Second constraint

In [ ]:
term_2_1 = Term(
    variable=x1,
    coefficient=5,
)
term_2_2 = Term(variable=x2, coefficient=2)
constraint_2 = Constraint(
    name='second constraint',
    left_hand_side=[term_2_1, term_2_2],
    sign=SignConstraint.LESSER_OR_EQUAL,
    right_hand_side=20,
)


All constraints.

In [ ]:
all_constraints = AllConstraints(
    constraints=[constraint_1, constraint_2]
)
print(all_constraints)


The definition of the problem gathers all the data defined above, and provides a name.

In [ ]:
the_problem: LinearProblem = LinearProblem(
    objective_coefficients=coefficients, constraints=all_constraints, the_name='P'
)


We apply the algorithm.

In [ ]:
optimal_solution = solve(a_subproblem=the_problem)
print(optimal_solution)



Note that the optimal solution was already found with the solution of P_1.
But it is only at the end of the algorithm that we know that it is actually the
optimal solution.